In [44]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from spacerocks.vector import Vector
from spacerocks.spacerock import SpaceRock
from spacerocks.utils import time_handler

import numpy as np
from astropy import units as u
from scipy.optimize import minimize

# Generate a state vector from spherical coordinates

In [137]:
def pointing_vector(theta, phi):
    return Vector(np.cos(theta) * np.cos(phi), np.cos(theta) * np.sin(phi), np.sin(theta))

def pointing_vector_rate(theta, phi, theta_rate, phi_rate):
    return Vector(-np.cos(theta) * np.sin(phi) * phi_rate - np.sin(theta) * np.cos(phi) * theta_rate, 
                   np.cos(theta) * np.cos(phi) * phi_rate - np.sin(theta) * np.sin(phi) * theta_rate, 
                   np.cos(theta) * theta_rate)

In [168]:
def rock_from_spherical(r, theta, phi, r_dot, theta_dot, phi_dot, epoch):
    position = r * pointing_vector(theta, phi)
    velocity = r * pointing_vector_rate(theta, phi, theta_dot, phi_dot) + r_dot * pointing_vector(theta, phi)

    x, y, z = position.x, position.y, position.z
    vx, vy, vz = velocity.x, velocity.y, velocity.z

    rock = SpaceRock(x=x, y=y, z=z, vx=vx, vy=vy, vz=vz, epoch=epoch, frame='J2000', origin='ssb')
    return rock

In [169]:
arrokoth = SpaceRock.from_horizons('Arrokoth')
arrokoth.change_frame('J2000')
obs = arrokoth.observe(spiceid='ssb')

theta = obs.dec.rad[0]
phi = obs.ra.rad[0]
theta_dot = obs.dec_rate.to(u.rad/u.day).value[0]
phi_dot = obs.ra_rate.to(u.rad/u.day).value[0]

pos = Vector(obs.x.au, obs.y.au, obs.z.au)
vel = Vector(obs.vx, obs.vy, obs.vz)

r = pos.norm[0]
r_dot = pos.dot(vel).value[0] / r

epoch = obs.epoch.utc.jd

In [375]:
r, r_dot, theta_dot, phi_dot

(43.061044336128425,
 -0.00010276196420543171,
 7.978156515351171e-06,
 6.507812513132712e-05)

In [317]:
def residual(psi, params):

    # where must the rock be at the reference epoch to be at the correct sky location at the observation epoch?

    theta, phi = psi
    r, r_dot, theta_dot, phi_dot, reference_epoch, image_epoch, image_ra, image_dec = params

    position = r * pointing_vector(theta, phi)
    velocity = r * pointing_vector_rate(theta, phi, theta_dot, phi_dot) + r_dot * pointing_vector(theta, phi)

    x, y, z = position.x, position.y, position.z
    vx, vy, vz = velocity.x, velocity.y, velocity.z

    rock = SpaceRock(x=x, y=y, z=z, vx=vx, vy=vy, vz=vz, epoch=reference_epoch, frame='J2000', origin='ssb')
    prop = rock.analytic_propagate(epoch=image_epoch)

    obs = prop.observe(spiceid='ssb')

    return np.sqrt((obs.ra.rad - image_ra)**2 + (obs.dec.rad - image_dec)**2) * u.rad.to(u.arcsec)

In [335]:
reference_epoch = time_handler('20 April 2023').utc.jd

image_epoch = epoch
image_ra = phi
image_dec = theta

In [373]:
params = [r, r_dot, theta_dot, phi_dot, reference_epoch, image_epoch, image_ra, image_dec]
res = minimize(residual, [0, np.pi], args=(params,), method='Powell', bounds=[(-np.pi/2, np.pi/2), (0, 2*np.pi)], tol=0.00000001)

In [374]:
res

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 0.0004165739771010905
       x: [-3.475e-01  5.116e+00]
     nit: 3
   direc: [[ 1.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00]]
    nfev: 161